In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_code(symbol):
    krx = pd.read_csv('./krx_code.csv',encoding='utf-8')
    krx = krx.set_index('한글 종목약명')
    try:
        code = krx.at[symbol,'단축코드']
        return code
    except:
        print('종목명을 다시 확인해주세요.')
        return 0

def get_comment_csv(symbol,page,date_point):   
    code = get_code(symbol)
    date_list = [] # 날짜
    comment_list = [] # 댓글
    view_list = [] # 조회수
    good_list = [] # 좋아요
    bad_list = [] # 싫어요
    for i in range(1,page+1):
        url = f'https://finance.naver.com/item/board.naver?code={code}&page={i}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'}
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text, 'html.parser')
        for j in range(20):
            try:
                root = bs.find('div',{'class':'section inner_sub'}).find_all('tr',{'onmouseover':'mouseOver(this)'})[j].text.split('\n')

                date_list.append(root[1].split()[0].replace('.','-'))
                
                if len(root) == 14: # 답글
                    comment_list.append('답글:'+root[4])
                    view_list.append(root[10])
                    good_list.append(root[11])
                    bad_list.append(root[12])          
                elif len(root) == 13: # 기본
                    comment_list.append(root[3])
                    view_list.append(root[9])
                    good_list.append(root[10])
                    bad_list.append(root[11])
                else: # 에러
                    comment_list.append('error')
                    view_list.append(0)
                    good_list.append(0)
                    bad_list.append(0)   
            except:
                date_list.append('error')
                comment_list.append('error')
                view_list.append(0)
                good_list.append(0)
                bad_list.append(0)   
        print(f'\r{i}페이지 크롤링 완료.',end='')
        
        if date_point in date_list:
                        break
            
    df = pd.DataFrame()
    df['날짜'] = date_list
    df['댓글'] = comment_list
    df['조회수'] = view_list
    df['좋아요'] = good_list
    df['싫어요'] = bad_list
    return df

In [3]:
pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)

,번호,종목명,ISIN,종목코드,수량,비중(%),평가금액(원),현재가(원),등락(원)
1,2,삼성전자,KR7005930003,005930,6110,0.218129,362934000,58000,-1400
2,3,NAVER,KR7035420009,035420,611,0.092723,154277500,246500,-6000
3,4,삼성SDI,KR7006400006,006400,244,0.085202,141764000,570000,-11000
4,5,LG화학,KR7051910008,051910,244,0.083589,139080000,530000,-40000
5,6,카카오,KR7035720002,035720,1222,0.052586,87495200,70800,-800
6,7,유한양행,KR7000100008,000100,1222,0.041202,68554200,55700,-400
7,8,SK텔레콤,KR7017670001,017670,1222,0.040027,66599000,53000,-1500
8,9,POSCO홀딩스,KR7005490008,005490,244,0.035635,59292000,234000,-9000
9,10,현대모비스,KR7012330007,012330,244,0.030283,50386000,199000,-7500
10,11,SK이노베이션,KR7096770003,096770,244,0.029549,49166000,200000,-1500


In [4]:
ktop30_company = pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']

In [8]:
df = get_comment_csv('삼성전자',500,'2022-06-30') # sample


433페이지 크롤링 완료.

In [9]:
df = get_comment_csv('미래에셋증권',500,'2022-06-30') # sample


1페이지 크롤링 완료.

In [6]:
df

,날짜,댓글,조회수,좋아요,싫어요
0,2022-07-01,월요일 종가를 알려드림,4,0,0
1,2022-07-01,영두 요즘도 회원받음?,20,0,0
2,2022-07-01,카카오 대표님,22,0,1
3,2022-07-01,카카오 공매도가 너무나 많네요,24,1,0
4,2022-07-01,개미님들 우리는 정부와 기관의 호구 먹잇...,25,1,0
...,...,...,...,...,...
635,2022-07-01,코스닥 500,98,0,0
636,2022-07-01,원래,98,0,0
637,2022-07-01,단기 코로나 수혜주 입니다.,245,0,1
638,2022-07-01,msci,202,2,0


In [5]:
for company in ktop30_company:
    df = get_comment_csv(company,77035,'2021-06-07')
    #df = df[df['날짜']=='2022-07-01']
    df.to_csv(f'./src/{company}_year.csv')

220페이지 크롤링 완료...

In [16]:
pd.read_csv('./src/year/삼성전자_year.csv')

,Unnamed: 0,날짜,댓글,조회수,좋아요,싫어요
0,0,2022-07-04,똥치운다 더니 더 똥싸는 종말 새끼 뭐하...,5,1,2
1,1,2022-07-04,토른방 글을 살펴보니~~~,16,2,1
2,2,2022-07-04,그냥 준stone이 대통령해라,8,0,4
3,3,2022-07-04,막상 상승장이 오더라도,7,0,2
4,4,2022-07-04,윤재앙 ㄱ 새끼 개같은 보수 정권,12,4,2
...,...,...,...,...,...,...
549175,549175,2021-06-07,이재용 대통령각하 ㅋ,190,4,1
549176,549176,2021-06-07,이재용 사면 외치는 놈들 제정신이냐?,222,3,6
549177,549177,2021-06-07,우리아버지가 개전자 9만에 팔고 삼성바이...,365,1,2
549178,549178,2021-06-07,성범죄가 많은이유 ////////////...,212,4,3
